In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures



cities_df = pd.read_csv('wojewodztwa_miasta.csv',sep=';')


        

In [2]:
from unidecode import unidecode

def remove_polish_chars(text):
    return unidecode(text)

# Zastosuj funkcję do kolumny 'nazwa miasta'
cities_df['nazwa miasta'] = cities_df['nazwa miasta'].apply(remove_polish_chars).str.lower()

cities = cities_df['nazwa miasta']

In [72]:
cities

0       boleslawiec
1      nowogrodziec
2           bielawa
3       dzierzoniow
4          pieszyce
           ...     
913           resko
914       wegorzyno
915        koszalin
916        szczecin
917     swinoujscie
Name: nazwa miasta, Length: 918, dtype: object

In [73]:
from bs4 import BeautifulSoup

def max_page_number(response):
    soup = BeautifulSoup(response.content, 'html.parser')
    pagination_links = soup.find_all('a', href=True)  # znajdź wszystkie linki z atrybutem href

    page_numbers = []
    for link in pagination_links:
        page_number = link.text.strip()  # wyciągnij tekst linku i usuń białe znaki z początku i końca
        if page_number.isdigit():  # sprawdź, czy tekst linku zawiera tylko cyfry
            page_numbers.append(int(page_number))  # dodaj numer podstrony do listy
            
    if page_numbers:
        return max(page_numbers)
    else:
        return 1

In [76]:
number = {}
i = 0
for city in cities:
    url = f'https://gratka.pl/nieruchomosci/mieszkania/{city}'
    response = requests.get(url, timeout=2)
    max_page = max_page_number(response)
    number[city] = max_page
    print(i,':',city,':',max_page)
    i+=1



0 : boleslawiec : 2
1 : nowogrodziec : 1
2 : bielawa : 1
3 : dzierzoniow : 1
4 : pieszyce : 1
5 : pilawa gorna : 1
6 : niemcza : 1
7 : glogow : 3
8 : gora : 1
9 : wasosz : 1
10 : jawor : 1
11 : bolkow : 1
12 : karpacz : 3
13 : kowary : 1
14 : piechowice : 1
15 : szklarska poreba : 1
16 : kamienna gora : 1
17 : lubawka : 1
18 : duszniki-zdroj : 1
19 : klodzko : 1
20 : kudowa-zdroj : 1
21 : nowa ruda : 1
22 : polanica-zdroj : 1
23 : bystrzyca klodzka : 1
24 : ladek-zdroj : 1
25 : miedzylesie : 1
26 : radkow : 1
27 : stronie slaskie : 1
28 : szczytna : 1
29 : chojnow : 1
30 : prochowice : 1
31 : luban : 1
32 : swieradow-zdroj : 2
33 : lesna : 1
34 : olszyna : 1
35 : lubin : 2
36 : scinawa : 1
37 : gryfow slaski : 1
38 : lubomierz : 1
39 : lwowek slaski : 1
40 : mirsk : 1
41 : wlen : 1
42 : milicz : 2
43 : olesnica : 1
44 : bierutow : 1
45 : miedzyborz : 1
46 : sycow : 1
47 : twardogora : 1
48 : olawa : 3
49 : jelcz-laskowice : 1
50 : chocianow : 1
51 : polkowice : 2
52 : przemkow : 1
53 :

In [77]:
df = pd.DataFrame(list(number.items()), columns=['miasto', 'numer_podstrony'])

In [78]:
df.to_csv('liczba_podstron_z_miastem.csv')

,miasto,numer_podstrony
0,boleslawiec,2
1,nowogrodziec,1
2,bielawa,1
3,dzierzoniow,1
4,pieszyce,1
...,...,...
911,resko,1
912,wegorzyno,1
913,koszalin,3
914,szczecin,35


In [79]:
df['miasto'][1]

'nowogrodziec'

In [ ]:
df['numer_podstrony'][1]

In [80]:
i=0
data_href_list = []
def gratka(response):
    soup = BeautifulSoup(response.content, 'html.parser')

# Znajdź wszystkie znaczniki article na stronie
    articles = soup.find_all('article', class_='teaserUnified')
    
# Iteruj przez znalezione artykuły
    for article in articles:
        data_href = article.get('data-href', '')    
        print(i+1,data_href)  
        data_href_list.append(data_href)
        


In [84]:
for i in range(len(df)):     
  for liczba in range(df['numer_podstrony'][i]):
      # Zakres stron do przeszukania
    url = f'https://gratka.pl/nieruchomosci/mieszkania/{cities[i]}?page={liczba}'
    response = requests.get(url, timeout=10)
    print(gratka(response))

1 https://gratka.pl/nieruchomosci/mieszkanie-boleslawiec/ob/33396539
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-boleslawiec-ul-plac-piastowski/ob/23365411
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-boleslawiec-ul-nadrzeczna/ob/33717249
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-boleslawiec-ul-staroszkolna/ob/33637483
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-boleslawiec-ul-staroszkolna/ob/33591693
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-boleslawiec-ul-staroszkolna/ob/33591393
1 https://gratka.pl/nieruchomosci/mieszkanie-boleslawiec-ul-k-i-galczynskiego/ob/33422961
1 https://gratka.pl/nieruchomosci/mieszkanie-boleslawiec-ul-malachowskiego/ob/33153075
1 https://gratka.pl/nieruchomosci/mieszkanie-boleslawiec-ul-gdanska/ob/31679597
1 https://gratka.pl/nieruchomosci/nowe-mieszkanie-boleslawiec-ul-staroszkolna/ob/30728755
1 https://gratka.pl/nieruchomosci/mieszkanie-boleslawiec-ul-wybickiego/ob/30546887
1 https://gratka.pl/nieruchomosci/mieszkanie-boles

In [4]:
#pd.DataFrame(data_href_list).to_csv('linki1.csv')

In [7]:
len(linki)

40839

In [5]:
linki = pd.read_csv('linki1.csv')

In [19]:
linki['0'][10:]

10       https://gratka.pl/nieruchomosci/mieszkanie-bol...
11       https://gratka.pl/nieruchomosci/mieszkanie-bol...
12       https://gratka.pl/nieruchomosci/mieszkanie-bol...
13       https://gratka.pl/nieruchomosci/mieszkanie-bol...
14       https://gratka.pl/nieruchomosci/mieszkanie-bol...
                               ...                        
40834    https://gratka.pl/nieruchomosci/mieszkanie-kos...
40835    https://gratka.pl/nieruchomosci/nowe-mieszkani...
40836    https://gratka.pl/nieruchomosci/mieszkanie-kos...
40837    https://gratka.pl/nieruchomosci/nowe-mieszkani...
40838    https://gratka.pl/nieruchomosci/mieszkanie-kos...
Name: 0, Length: 40829, dtype: object

In [13]:
data_dict = {}
max_threads = 6
with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
    def gratka1(response):
        soup = BeautifulSoup(response.content, 'html.parser')

    # Znalezienie wszystkich elementów li w obrębie listy
    
        try:
            data_dict['region'] = soup.select_one('.offerLocation__region').text.strip()
        except AttributeError:
            data_dict['region'] = ''

        try:
            data_dict['title'] = soup.select_one('.sticker__title').text.strip()
        except AttributeError:
            data_dict['title'] = ''

        try:
            data_dict['total_price'] = soup.select_one('.priceInfo__value').text.strip()
        except AttributeError:
            data_dict['total_price'] = ''

        try:
            data_dict['additional_price_per_m2'] = soup.select_one('.priceInfo__additional').text.strip()
        except AttributeError:
            data_dict['additional_price_per_m2'] = ''

        try:
            opis = soup.find('div', class_='description__rolled ql-container').text.strip()
        except AttributeError:
            opis = ''
        data_dict['opis'] = opis
    # Iteracja po każdym elemencie li
        li_elements = soup.select('.parameters__singleParameters li')
        for li in li_elements:
            span = li.find('span')
            value = li.find('b', class_='parameters__value')

            if span and value:
                span_text = span.text.strip()
                value_text = value.text.strip()
                data_dict[span_text] = value_text
            
        li_elements1 = soup.select('.parameters__groupedParameters li')
        for li1 in li_elements1:
            liw = li1.get('data-cy')
            lit = li1.text.strip()
            data_dict[liw] = lit
        data_dict['opis'] = opis
    # Znajdź wszystkie tagi img w obrębie div.gallery__container
        img_elements = soup.select('div.gallery__container img')

# Pobierz atrybut src z każdego tagu img
        src_values = [img['src'] for img in img_elements]
        combined_src = ' '.join(src_values)
        data_dict['links_pictures'] = combined_src
        data = pd.DataFrame(data_dict, index=[0])
        return data 


In [27]:
'''import pandas as pd

# Utwórz pustą ramkę danych przed pętlą
df1 = pd.DataFrame()

for url in linki['0']:
    response = requests.get(url)
    if response.status_code == 200:
        a = gratka1(response)
        
        # Połącz wynik z ramką danych w każdej iteracji
        df1 = pd.concat([df1, pd.DataFrame(a)], ignore_index=True)
        
    else:
        print(f"Nieudane żądanie dla URL: {url}")

# Wyświetl zaktualizowaną ramkę danych
print(df1)
'''
import pandas as pd
import time

# Utwórz pustą ramkę danych przed pętlą
df1 = pd.DataFrame()
i=0
for url in linki['0'][9978:]:
    try:
        response = requests.get(url)
        response.raise_for_status()  # Sprawdź, czy wystąpił błąd HTTP
        
        if response.status_code == 200:
            a = gratka1(response)
        
            # Połącz wynik z ramką danych w każdej iteracji
            df1 = pd.concat([df1, pd.DataFrame(a)], ignore_index=True)
            
        else:
            print(f"Nieudane żądanie dla URL: {url}")
    except requests.exceptions.RequestException as e:
        print(f"Błąd podczas pobierania danych z URL: {url}")
        print(e)  # Wyświetl szczegóły błędu
    
    # Dodaj opóźnienie 1 sekundy między żądaniami
    time.sleep(3)
    print(i+=1)

# Wyświetl zaktualizowaną ramkę danych
print(df1)



1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 

In [28]:
df1

,region,title,total_price,additional_price_per_m2,opis,Lokalizacja,Powierzchnia w m2,Liczba pokoi,Piętro,Forma własności,...,"Opłaty (czynsz administracyjny, media)",Usytuowanie względem stron świata,prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro
0,"Wrocław,","Mieszkanie Wrocław Biskupin, ul. Norblina",550 000\n \n zł,"11 956,52 zł/m2",Opiekun ofertyDorota tel: \n\n+48...\npokaż nu...,"Wrocław, ...",46 m2,2,2,własność,...,495 zł,"północne, południowe",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro
1,"Wrocław,",Mieszkanie Wrocław Gaj,550 000\n \n zł,"11 684,72 zł/m2",MIESZKANIE DO WPROWADZENIA / STAN IDEALNY DWA ...,"Wrocław, ...","47,07 m2",2,1,własność,...,495 zł,"północne, południowe",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro
2,"Wrocław,","Mieszkanie Wrocław Brochów, ul. i. Mościckiego...",450 000\n \n zł,"7 729,30 zł/m2",PKP S.A. Oddział Gospodarowania Nieruchomościa...,"Wrocław, ...","58,22 m2",2,parter,własność,...,495 zł,"północne, południowe",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro
3,"Wrocław,","Nowe mieszkanie Wrocław Śródmieście, ul. Sienk...",552 060\n \n zł,18 000 zł/m2,INTERESUJĄCA INWESTYCJA W CENTRUM Z WIDOKIEM N...,"Wrocław, ...","30,67 m2",2,2,własność,...,495 zł,"północne, południowe",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro
4,"Wrocław,","Mieszkanie Wrocław Przedmieście Oławskie, ul. ...",2 099 000\n \n zł,"15 321,17 zł/m2",5 MIKROAPARTAMENTÓW TYLKO 10 MINUT PIESZO DO R...,"Wrocław, ...",137 m2,5,1,własność,...,495 zł,"północne, południowe",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,"Bydgoszcz,",Mieszkanie Bydgoszcz Wzgórze Wolności,599 000\n \n zł,"8 940,30 zł/m2",Biuro Jagła Nieruchomości zaprasza do zapozna...,"Bydgoszcz, ...",67 m2,3,4,własność,...,400 zł,"północne, południowe",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro
1645,"Bydgoszcz,","Mieszkanie Bydgoszcz Wyżyny, ul. Adama Grzymał...",450 000\n \n zł,"8 609,14 zł/m2","Szukasz przestronnego, wykończonego, gotowego ...","Bydgoszcz, ...","52,27 m2",3,2,własność,...,400 zł,"północne, południowe",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro
1646,"Bydgoszcz,","Mieszkanie Bydgoszcz Okole, ul. Grunwaldzka",259 000\n \n zł,7 000 zł/m2,"Szanowni Państwo,Z przyjemnością prezentujemy ...","Bydgoszcz, ...",37 m2,2,2,własność,...,400 zł,"północne, południowe",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro
1647,"Bydgoszcz,","Mieszkanie Bydgoszcz, ul. Akademicka",482 237\n \n zł,"8 850,01 zł/m2","Marzysz o mieszkaniu w dynamicznym miejscu, tu...","Bydgoszcz, ...","54,49 m2",3,4,własność,...,400 zł,"północne, południowe",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro


In [29]:
df1.to_csv('mieszkanka1648.csv')

In [30]:
# Przykładowa ramka danych


# Dzielimy każdy adres po słowie "ul."
df1['Część_po_ul'] = df1['title'].str.split('ul.').str.get(1).str.strip()

# Wyświetlamy ramkę danych z nową kolumną
df1['Część_po_ul']

0                          Norblina
1                               NaN
2              i. Mościckiego 32/2a
3                      Sienkiewicza
4                      Krasińskiego
                   ...             
1644                            NaN
1645    Adama Grzymały Siedleckiego
1646                    Grunwaldzka
1647                     Akademicka
1648                            NaN
Name: Część_po_ul, Length: 1649, dtype: object

In [31]:
df1['Lokalizacja']

0       Wrocław,                                      ...
1       Wrocław,                                      ...
2       Wrocław,                                      ...
3       Wrocław,                                      ...
4       Wrocław,                                      ...
                              ...                        
1644    Bydgoszcz,                                    ...
1645    Bydgoszcz,                                    ...
1646    Bydgoszcz,                                    ...
1647    Bydgoszcz,                                    ...
1648    Bydgoszcz,                                    ...
Name: Lokalizacja, Length: 1649, dtype: object